color_argument2 algorithm explanation (return True means contradiction):

    takes in: current vertex v_0, done_list, current_distance, max_distance, max_depth
    Make color_list = (0, 1, 2) - color(v_0) (if v_0) is colored
    
    If current_distance > 1 and you've looped back to the start, return False
    Alternatively, if current_distance >= max_distance, return False (do not proceed). 
    
    Supposing neither of these is the case, for each color c_0 in color_list:
    
        make 5 lists:
    
            list0: Check all the neighbors of v_0. If v_1 in N(v_0) is colored with c_0, run color_argument2(v_1, done_list + v_0, current_distance +1, max_distance, max_depth). If False (no contradiction) then append v_1 to list0. 

            list1: Check all the neighbors of v_0 again. If v_1 in N(v_0) is colored with 'None', check if it has any neighbors colored with c_0. If not, make a copy of the graph G', and within G' color v_1 with c_0. Run the methods that extend colorings appropriately through the graph, and check for forks or triangles. If these methods indicate no contradiction, run color_argument2(v_1, done_list + v_0, current_distance +1, max_distance, max_depth). If False (no contradiction) then append v_1 to list1. 

            list2: Check all vertices in the 'unknown' list of v_0. If v_1 in the 'unknown' list of v_0 has no neighbors colored c_0, make a copy of the graph G', and within G' color v_1 with c_0 and add an edge from v_0 to v_1. Run the methods that extend colorings appropriately through the graph, and check for forks or triangles. If these methods indicate no contradiction, run color_argument2(v_1, done_list + v_0, current_distance +1, max_distance, max_depth). If False (no contradiction) then append v_1 to list2.

            list3: If v_0 is not max degree, then make a copy G' of the graph. add a new vertex v_1 to G' and color it with c_0 and add an edge between v_1 and v_0. Run the methods that extend colorings appropriately through the graph, and check for forks or triangles. If these methods indicate no contradiction, run color_argument2(v_1, done_list + v_0, current_distance +1, max_distance, max_depth). If False (no contradiction) then append v_1 to list3.

            list4: Check all vertices in the 'unknown' list of v_0 again. If v_1 in the 'unknown' list of v_0 is colored with c_0, make a copy of the graph G', and within G' add an edge from v_0 to v_1. Run the methods that extend colorings appropriately through the graph, and check for forks or triangles. If these methods indicate no contradiction, run color_argument2(v_1, done_list + v_0, current_distance +1, max_distance, max_depth). If False (no contradiction) then append v_1 to list4.
        
        Check how many elements are in these lists:
            If all the lists are empty (for any color) return True
            If exactly one list is nonempty and has exactly 1 element, then add the according structure (NOTE: this is done on the current graph- in the case of recursion, this may be done on a copy, not the original):
                If list1 is nonempty, color the vertex of that index with c_0
                If list2 is nonempty, add an edge from v_0 to the vertex of that index and color that vertex with c_0
                If list3 is nonempty, add a new vertex to the graph adjacent to v_0 of color c_0
                If list4 is nonempty, add an edge from v_0 to the vertex of that index
                
    return False (i.e., the default is to return False unless something prompts us to return 'True')
       
        

In [130]:
import copy
from itertools import permutations

class vert: #####The class of vertices #####
    def __init__(self, edges, non_edges, index):
        self.edges = edges #####List of indices of vertices adjacent to this vertex #####
        self.non_edges = non_edges ##### List of indices of vertices with a non-edge to this vertex #####
        self.index= index #####label unique to each vertex #####
        self.max_degree=None #####max degree, can be assigned to vertices later#####
        self.color=None ##### Color assigned to this vertex, can be assigned later #####
        self.color_start=False ##### This is set to 'True' if a partial coloring of G originates around this vertex being unable to be colored #####
        
class graph:
    def __init__(self, vertex_list):
        self.vertex_list=vertex_list ##### List of vertex objects #####
        self.index_list=[vertex.index for vertex in self.vertex_list] ##### List of indices of the vertices #####
        self.depth=0 ##### Used in methods later #####
        
    def show(self): ##### Method to display the graph #####
        for vertex in self.vertex_list:
            print('Vertex ' + str(vertex.index) + ':')
            print('    Edges:     ' + str(vertex.edges))
            print('    Non-edges: ' + str(vertex.non_edges))
            print('    Unknown:   ' + str(vertex.unknown))
            print('    Color:   ' + str(vertex.color))
            
    def contains_triangle(self): #####Simple method that returns True if the graph contains a triangle, and False otherwise #####
        for v_0 in self.vertex_list:
            edge_lists = list(permutations(v_0.edges, 2))
            for temp_list in edge_lists:
                for vertex in self.vertex_list:
                    if vertex.index ==temp_list[0]:
                        v_1=vertex
                    if vertex.index ==temp_list[1]:
                        v_2=vertex  
                if (v_1.index != v_2.index) and ((v_1.index in v_2.edges) or (v_2.index in v_1.edges)):
                    #print('triangle: ' + str(v_0.index) + str(v_1.index) + str(v_2.index))
                    return True
        return False
    
    def is_fork(self): #####This method checks if the current graph is a fork (order is important) #####
        if len(self.vertex_list) !=7:
            return False
        if ((self.vertex_list[1].index in self.vertex_list[0].edges) and
            (self.vertex_list[2].index in self.vertex_list[0].edges) and
            (self.vertex_list[3].index in self.vertex_list[0].edges) and
            (self.vertex_list[4].index in self.vertex_list[0].edges) and
            (self.vertex_list[5].index in self.vertex_list[1].edges) and
            (self.vertex_list[6].index in self.vertex_list[2].edges) and
            (self.vertex_list[0].index in self.vertex_list[5].non_edges) and 
            (self.vertex_list[0].index in self.vertex_list[6].non_edges) and 
            (self.vertex_list[1].index in self.vertex_list[2].non_edges) and 
            (self.vertex_list[1].index in self.vertex_list[3].non_edges) and 
            (self.vertex_list[1].index in self.vertex_list[4].non_edges) and 
            (self.vertex_list[1].index in self.vertex_list[6].non_edges) and 
            (self.vertex_list[2].index in self.vertex_list[3].non_edges) and 
            (self.vertex_list[2].index in self.vertex_list[4].non_edges) and 
            (self.vertex_list[2].index in self.vertex_list[5].non_edges) and 
            (self.vertex_list[3].index in self.vertex_list[4].non_edges) and 
            (self.vertex_list[3].index in self.vertex_list[5].non_edges) and 
            (self.vertex_list[3].index in self.vertex_list[6].non_edges) and 
            (self.vertex_list[4].index in self.vertex_list[5].non_edges) and 
            (self.vertex_list[4].index in self.vertex_list[6].non_edges) and 
            (self.vertex_list[5].index in self.vertex_list[6].non_edges)):
            return True
        return False
        
    def contains_fork(self): #####This method checks the graph for forks by looking at permutations of the known edges and non-edges of each vertex in the graph. #####
        for vertex1 in self.vertex_list: #####Vertex1 is supposed to be the center of the fork
            temp_edge_lists = list(permutations(vertex1.edges, 4)) #####Need 4 vertices adjacent to the center of the fork
            temp_non_edge_lists = list(permutations(vertex1.non_edges, 2)) ##### and 2 that are not. Ensuring this structure in advance improves the runtime substantially.
            for temp_edge_list in temp_edge_lists:
                for temp_non_edge_list in temp_non_edge_lists:
                    #####Make a list of the vertices to make up the fork #####
                    temp_vert_list=[vertex1]
                    #####Append the edges #####
                    for index in temp_edge_list:
                        for vertex in self.vertex_list:
                            if vertex.index==index:
                                temp_vert_list.append(vertex)
                    #####And non-edges #####
                    for index in temp_non_edge_list:
                        for vertex in self.vertex_list:
                            if vertex.index==index:
                                temp_vert_list.append(vertex)
                    #####Instantiate as a graph object, and check if it is a fork #####
                    temp_graph=graph(temp_vert_list)
                    if temp_graph.is_fork():
                        return True
        return False
    
    def clean(self): 
        #####This method fixes the edge/non-edge lists of the graph. #####
        #####If v1 has v2 in its edge list, but v2 doesn't have v1 in its edge list, then clean will append v1 to v2's edge list. #####
        #####After all lists have been updated, it will make an 'unknown' list for each vertex, of neither edges nor non-edges. #####
        #####It is recommended to call this method after making changes to a graph. ##### '
        self.index_list=[vertex.index for vertex in self.vertex_list]
        keep_going = True
        while keep_going:
            #####Make a copy so you can check if the graph was updated #####
            old_graph=copy.deepcopy(self)
            for vertex in self.vertex_list:
                #####Making sure edge lists match up #####
                for vertex2_index in vertex.edges:
                    for vertex2 in self.vertex_list: 
                        if (vertex2.index==vertex2_index):
                            if vertex.index not in vertex2.edges:
                                vertex2.edges.append(vertex.index)
                #####Making sure non-edge lists match up #####
                for vertex2_index in vertex.non_edges:
                    for vertex2 in self.vertex_list:
                        if (vertex2.index==vertex2_index):
                            if vertex.index not in vertex2.non_edges:
                                vertex2.non_edges.append(vertex.index)
                #####If any vertex becomes max degree, we prohibit any further edges for that vertex#####
                if len(vertex.edges)==vertex.max_degree:
                    vertex.non_edges = list(set(self.index_list) - set(vertex.edges))
            #####Now we check if the graph was updated#####
            updated=False
            for vertex1 in self.vertex_list:
                for vertex2 in old_graph.vertex_list:
                    if ((vertex1.index == vertex2.index) and ((vertex1.edges != vertex2.edges) or (vertex1.non_edges !=vertex2.non_edges))):
                        updated=True
            if updated==False:
                keep_going=False
        #####Once all edge/non-edge lists are updated, we set the unknown lists for each vertex #####
        for vertex in self.vertex_list:
            vertex.unknown = list(set(self.index_list).difference(vertex.edges + vertex.non_edges + [vertex.index]))
    
    def check_for_forks_triangles(self, depth_cutoff):
        #####This method scans the graph for forks and triangles, as well as seeing if appending certain edges/nonedges is forced by forks or triangles. #####
        #####Will test all combinations of n edges/non-edges, where n is the depth_cutoff#####
        #####In practice, little is gained by a depth cutoff of more than one#####
        #####Method returns True if a fork/triangle is forced, False otherwise #####
        if (self.contains_fork()):
            #print('There is a fork!')
            return True
        if (self.contains_triangle()):
            #print('There is a triangle!')
            return True
        
        if self.depth < depth_cutoff:
            for vertex in self.vertex_list:
                if len(vertex.unknown) > 0:
                    for edge in vertex.unknown:
                        if edge > vertex.index: #####Prevents running each case twice, to cut down the runtime #####
                            #####Make a temporary graph with the appended edge#####
                            temp_graph = copy.deepcopy(self)
                            for temp_vert in temp_graph.vertex_list:
                                if temp_vert.index == vertex.index:
                                    temp_vert.edges.append(edge)
                            temp_graph.clean()
                            temp_graph.depth=self.depth+1 #####Need to increase the depth, so it doesn't recurse forever

                            #####Make another temporary graph with the appended non-edge #####
                            temp_graph2 = copy.deepcopy(self)
                            for temp_vert in temp_graph2.vertex_list:
                                if temp_vert.index == vertex.index:
                                    temp_vert.non_edges.append(edge)
                            temp_graph2.clean()
                            temp_graph2.depth=self.depth+1 ####Need to increase the depth, so it doesn't recurse forever
                            
                            ##### Check both cases. If there is a contradiction in either case, return True ##### 
                            bool1 = temp_graph.check_for_forks_triangles(depth_cutoff)
                            bool2 = temp_graph2.check_for_forks_triangles(depth_cutoff)
                            if (bool1 and bool2):
                                return True
                            ##### In this case, only and edge leads to a fork/triangle, so we append a non-edge #####
                            elif bool1:
#                                 print('non-edge added!')
                                vertex.non_edges.append(edge)
                                self.clean()
                            ##### In this case, an edge is added #####
                            elif bool2:
#                                 print('edge added!')
                                vertex.edges.append(edge)
                                self.clean()
        return False

    def check_for_triangles(self):
        #####This method scans the graph for triangles, as well as seeing if appending certain non-edges is forced by triangles. #####
        #####Method returns True if a fork/triangle is forced, False otherwise #####
        if (self.contains_triangle()):
            #print('There is a triangle!')
            return True

        for vertex in self.vertex_list:
            if len(vertex.unknown) > 0:
                for edge in vertex.unknown:
                    if edge > vertex.index: #####Prevents running each case twice, to cut down the runtime #####
                        #####Make a temporary graph with the appended edge#####
                        temp_graph = copy.deepcopy(self)
                        for temp_vert in temp_graph.vertex_list:
                            if temp_vert.index == vertex.index:
                                temp_vert.edges.append(edge)
                        temp_graph.clean()
                        
                        if temp_graph.contains_triangle():
                            vertex.non_edges.append(edge)
                            self.clean()
        return False
    
    def get_fork_edge_lists(self):
        #####This method returns a list of lists of edges #####
        #####Each list of edges corresponds to a fork, and one of the edges listed must exist to prevent the fork from existing in the graph #####
        #####This method assumes the graph has no forks in it- if it does, one of the lists may be empty #####
        temp_list = []
        for vertex0 in self.vertex_list:
            temp_edge_lists = list(permutations(vertex0.edges, 4)) #####Need 4 vertices adjacent to the center of the fork
            for edge_list in temp_edge_lists:
                #####Prevents doing each case twice #####
                if (edge_list[0] > edge_list[1]) or (edge_list[2] > edge_list[3]): 
                    continue
                #####Find Next 4 vertices of fork #####
                for temp_vertex in self.vertex_list:
                    if temp_vertex.index==edge_list[0]:
                        vertex1 = temp_vertex
                    if temp_vertex.index==edge_list[1]:
                        vertex2 = temp_vertex
                    if temp_vertex.index==edge_list[2]:
                        vertex3 = temp_vertex
                    if temp_vertex.index==edge_list[3]:
                        vertex4 = temp_vertex
                #print(edge_list)
                #####Find suitable candidates for sixth vertex of fork
                for edge1 in vertex1.edges:
                    vertex5=None
                    for vertex in self.vertex_list:
                        if (vertex.index == edge1) and (edge1 != vertex0.index):
                            if (edge_list[1] not in vertex.edges) and (edge_list[2] not in vertex.edges) and (edge_list[3] not in vertex.edges):
                                vertex5= vertex
                    #####For control flow reasons, continue unless a vertex5 was chosen #####
                    if vertex5 == None:
                        continue
                    #print(vertex0.index, vertex1.index, vertex2.index, vertex3.index, vertex4.index, vertex5.index)
                    
                    for edge2 in vertex2.edges:
                        vertex6=None
                        for vertex in self.vertex_list:
                            if (vertex.index == edge2) and (edge2 != vertex0.index):
                                if (edge_list[0] not in vertex.edges) and (edge_list[2] not in vertex.edges) and (edge_list[3] not in vertex.edges) and (vertex5.index not in vertex.edges):
                                    vertex6= vertex
                        if vertex6 == None:
                            continue
                        
                    
                        #####now make the list of ways to avoid the fork #####
                        more_temp_list = []
                        for index in edge_list[1:]:
                            if index in vertex5.unknown:
                                more_temp_list.append([index, vertex5.index])
                        temp_indices=[edge_list[0], edge_list[2], edge_list[3], vertex5.index]
                        for index in temp_indices:
                            if index in vertex6.unknown:
                                more_temp_list.append([index, vertex6.index])
                        temp_list.append(more_temp_list) 
        return temp_list
    
    def check_for_forks_triangles_iter(self, depth_cutoff):
        #####This method runs the prior check_for_forks_triangles method iteratively until the graph is no longer updated #####
        #####Like most methods, this returns True if there is a contradiction and False otherwise #####
        for i in list(set([1, depth_cutoff])): #####For runtime reasons, I set the script to run at depth 1 first, then at the specified depth. Now, though, I usually just run the code with depth 1, so not sure this should be used. 
            keep_going = True
            while keep_going:
                old_graph = copy.deepcopy(self)
                bool1=self.check_for_forks_triangles(i)
                if bool1:
                    return True                
                updated=False
                for j, vertex in enumerate(old_graph.vertex_list):
                    if (vertex.edges != self.vertex_list[j].edges) or (vertex.non_edges != self.vertex_list[j].non_edges): 
                        updated = True
                if updated == False:
                    keep_going=False
        return False

    def fill_in_graph(self, depth_cutoff):
        #####This method runs check_for_forks_triangles_iter, as well as appending new neighbors where there is neighborhood containment. #####
        #####I don't really use it currently, though, so I'm going to to go light on annotating it #####
        keep_going = True
        while keep_going:
            old_graph = copy.deepcopy(self)
            ##### Run check_for_forks_triangles_iter and check for updates #####
            bool1 = self.check_for_forks_triangles_iter(depth_cutoff)
            if bool1:
                #print('Contradiction!')
                return True
            updated=False
            for j, vertex in enumerate(old_graph.vertex_list):
                if (vertex.edges != self.vertex_list[j].edges) or (vertex.non_edges != self.vertex_list[j].non_edges): 
                    updated = True
                    
            ##### Append vertices if there is neighborhood containment ######
            for vertex1 in self.vertex_list:
                for vertex2 in self.vertex_list:
                    if ((vertex1.index != vertex2.index) and (len(vertex1.unknown) ==0) and (set(vertex1.edges).issubset(set(vertex2.edges)))):
                        if (vertex1.max_degree == None) or ((vertex1.max_degree != None) and (len(vertex1.edges) < vertex1.max_degree)):
                            new_vertex=vert([vertex1.index], [vertex2.index], max(self.index_list) + 1)
                            self.vertex_list.append(new_vertex)
                            self.clean()
                            #print('New vertex added!')
                            updated=True
                        elif ((vertex1.max_degree != None) and (len(vertex1.edges) >= vertex1.max_degree)):
                            #print('Contradiction!')
                            return True        
            if updated == False:
                keep_going=False
        return False
    
    def clean_colors(self): #####Adds non-edges according to the coloring of the graph #####
        for color in [0, 1, 2]:
            index_list=[]
            last_vert = None
            for vertex in self.vertex_list:
                if vertex.color==color:
                    vertex.non_edges = list(set(vertex.non_edges + index_list))
                    index_list.append(vertex.index)
        self.clean()
        
    def color_vert(self, start_vert):
        #####This method attempts to color a single vertex #####
        color_list = []
        uncolored_neighbors = []
        for vertex in self.vertex_list:
            if vertex.index == start_vert:
                start_vertex=vertex
        
        #####Make the list of colors of neighbors, and indices of uncolored neighbors ########
        for neighbor in start_vertex.edges:
            for vert in self.vertex_list:
                if vert.index==neighbor:
                    if vert.color != None:
                        color_list.append(vert.color)
                    else:
                        uncolored_neighbors.append(vert.index)
        color_list = list(set(color_list))
        
        ######Color start_vertex, if uncolored #########
        if (len(color_list) == 3) and (start_vertex.color_start==False):
            #print('Contradiction!')
            return True
        if (len(color_list) == 2) and (start_vertex.color_start==False):
            if start_vertex.color == None:
                start_vertex.color = list(set([0, 1, 2]) - set(color_list))[0]
            elif (start_vertex.color != list(set([0, 1, 2]) - set(color_list))[0]):
                #print('Contradiction!')
                return True
        
        #######If it is the first vertex being colored and it is degree 3, color all its neighbors with different colors #######
        if start_vertex.color_start==True:
            if (start_vertex.max_degree ==3):
                i=0
                for neighbor in start_vertex.edges:
                    for vertex in self.vertex_list:
                        if vertex.index==neighbor:
                            vertex.color=i
                            i=i+1
        
        #######If it is not the starting vertex and a neighbor must be a certain color to force it's own color, then assign a color to that neighbor ##########
        if (start_vertex.max_degree != None) and (start_vertex.color_start==False):
            if len(start_vertex.edges) == start_vertex.max_degree and (len(uncolored_neighbors) == 1):
                if start_vertex.color !=None:
                    color_list.append(start_vertex.color)
                    color_list = list(set(color_list))
                if len((set([0, 1, 2]) - set(color_list)))>1:
                    #print('Contradiction!')
                    return True
                if len((set([0, 1, 2]) - set(color_list)))==1:
                    for vertex in self.vertex_list:
                        if vertex.index == uncolored_neighbors[0]:
                            vertex.color = list(set([0, 1, 2]) - set(color_list))[0]
        
        ########Fix up the graph (non-edges) after adding new colors #########
        self.clean_colors()
        return False
    
    def fill_in_colors(self, start_vert=None):
        #####This method fills in a partial coloring of the graph, starting from start_vert, and assuming that start_vert cannot be colored.#####
        #####can be run with start_vert=None to just update colors for the graph #######
        #####Color neighborhood of starting vertex ##########
        if start_vert !=None:
            for vertex in self.vertex_list:
                if vertex.index==start_vert:
                    vertex.color_start = True
                    if self.color_vert(start_vert)==True:
                        return True
        
        ######Iteratively add colors for the rest of the graph ########
        keep_going =True
        while keep_going==True:
            old_graph = copy.deepcopy(self)
            #####Color the vertex while also checking for a contradiction #####
            for vertex in self.vertex_list:
                if self.color_vert(vertex.index)==True:
                    return True
            #####Check for updates
            updated =False
            for vertex1 in self.vertex_list:
                for vertex2 in old_graph.vertex_list:
                    if (vertex1.index==vertex2.index) and (vertex1.color !=vertex2.color):
                        updated=True
            if updated==False:
                keep_going=False
        return False
    
    def color_argument2(self, start_vert, done_list, current_distance, max_distance, max_depth):
        #####This method recursively builds out new structure to the graph #####
        ##### done_list is the set of vertices already visited by color_argument2 within the recursion ######
        ##### current_distance is the recursion depth/distance in the graph from the starting vertex
        ##### max_distance is the max distance allowed within the graph
        ##### max_depth is the max recursion depth to which the graph is allowed to be modified 
        for vertex in self.vertex_list:
            if vertex.index==start_vert:
                start_vertex=vertex
        #####color_list is colors to be forced #####
        color_list = list(set([0, 1, 2]) - set([start_vertex.color]))
        
        #####It leads to algorithmic errors if you don't check that you haven't looped back to the start######
        if current_distance > 1:
            for edge_index in start_vertex.edges:
                for vertex1 in self.vertex_list:
                    if (vertex1.index == edge_index) and (vertex1.color_start ==True):
                        return False

        #####Check for each color that it exists among neighbors#########
        if (current_distance < max_distance) and (self.depth < max_depth):
            for temp_color in color_list:
                #####Check for already colored neighbors#####
                list0 = []
                for edge_index in start_vertex.edges:
                    for vertex1 in self.vertex_list:
                        if (vertex1.index==edge_index) and (vertex1.color == temp_color) and (vertex1.index not in done_list):
                            temp_done_list = done_list + [start_vert]
                            temp_graph = copy.deepcopy(self)
                            temp_graph.depth= self.depth+1
                            #####Recurse on vertex1
                            bool1=temp_graph.color_argument2(vertex1.index, temp_done_list, current_distance + 1, max_distance, max_depth)
                            if bool1==False:
                                list0.append(vertex1.index)

                #####Check for uncolored neighbors that could be assigned that color#####
                list1=[]
                for edge_index in start_vertex.edges:
                    for vertex1 in self.vertex_list:
                        if (vertex1.index==edge_index) and (vertex1.color == None) and (vertex1.index not in done_list):
                            #####Check that vertex1 can be colored with temp_color #####
                            neighbor_colors = []
                            for edge_index2 in vertex1.edges:
                                for vertex2 in self.vertex_list:
                                    if vertex2.index ==edge_index2:
                                        neighbor_colors.append(vertex2.color)
                            if temp_color not in neighbor_colors:
                                #####Copy graph and color vertex #####
                                temp_graph = copy.deepcopy(self)
                                temp_graph.depth= self.depth+1
                                for vertex3 in temp_graph.vertex_list:
                                    if vertex3.index==vertex1.index:
                                        vertex3.color=temp_color
                                #####Fill in colors and check for forks/triangles #####
                                bool1=temp_graph.fill_in_colors()
                                bool2=temp_graph.check_for_forks_triangles_iter(1)
                                #####Assuming no contradiction, recurse on vertex1#####
                                if not (bool1 or bool2):
                                    temp_done_list = done_list + [start_vert]
                                    bool3=temp_graph.color_argument2(vertex1.index, temp_done_list, current_distance + 1, max_distance, max_depth)
                                    if bool3==False:
                                        list1.append(vertex1.index)
                
                #####Check for potential neighbors that could be that color#####
                list2=[]
                for edge_index in start_vertex.unknown:
                    for vertex1 in self.vertex_list:
                        if (vertex1.index==edge_index) and (vertex1.color == None) and (vertex1.index not in done_list):
                            #####Check that it can be assigned temp_color #####
                            neighbor_colors = []
                            for edge_index2 in vertex1.edges:
                                for vertex2 in self.vertex_list:
                                    if vertex2.index ==edge_index2:
                                        neighbor_colors.append(vertex2.color)
                            if temp_color not in neighbor_colors:
                                #####Copy graph and add color and the new edge #####
                                temp_graph = copy.deepcopy(self)
                                temp_graph.depth= self.depth+1
                                for vertex3 in temp_graph.vertex_list:
                                    if vertex3.index==vertex1.index:
                                        vertex3.color=temp_color
                                        vertex3.edges.append(start_vert)
                                #####Clean the graph structure now that new structure has been added #####
                                temp_graph.clean()
                                bool1=temp_graph.fill_in_colors()
                                bool2=temp_graph.check_for_triangles()
                                #####Assuming no contradiction, recurse on vertex1 #####
                                if not (bool1 or bool2):
                                    temp_done_list = done_list + [start_vert]
                                    bool3=temp_graph.color_argument2(vertex1.index, temp_done_list, current_distance + 1, max_distance, max_depth)
                                    if bool3==False:
                                        list2.append(vertex1.index)

                #####Check if a new vertex of that color could be added#####
                list3=[]
                if len(start_vertex.edges) != start_vertex.max_degree:
                    ##### Copy graph and add vertex #####
                    temp_graph = copy.deepcopy(self)
                    temp_graph.depth= self.depth+1
                    temp_vert = vert([start_vert], [], max(self.index_list) + 1)
                    temp_vert.color=temp_color
                    temp_graph.vertex_list.append(temp_vert)
                    #####Clean the graph #####
                    temp_graph.clean()
                    bool1=temp_graph.fill_in_colors()
                    bool2=temp_graph.check_for_triangles()
                    #####Assuming no contradiction, recurse on temp_vert #####
                    if not (bool1 or bool2):
                        temp_done_list = done_list + [start_vert]
                        bool3=temp_graph.color_argument2(temp_vert.index, temp_done_list, current_distance + 1, max_distance, max_depth)
                        if bool3==False:
                            list3.append(temp_vert.index)
                            
                #####Check for potential neighbors that are already that color (poor order but I forgot and came back to this)#####
                list4=[]
                for edge_index in start_vertex.unknown:
                    for vertex1 in self.vertex_list:
                        if (vertex1.index==edge_index) and (vertex1.color == temp_color) and (vertex1.index not in done_list):
                            #####Copy graph and add the edge #####
                            temp_graph = copy.deepcopy(self)
                            temp_graph.depth= self.depth+1
                            for vertex3 in temp_graph.vertex_list:
                                if vertex3.index==vertex1.index:
                                    vertex3.edges.append(start_vert)
                            #####Clean the graph #####
                            temp_graph.clean()
                            bool1=temp_graph.fill_in_colors()
                            bool2=temp_graph.check_for_triangles()
                            #####Assuming no contradiction, recurse on vertex1 #####
                            if not (bool1 or bool2):
                                temp_done_list = done_list + [start_vert]
                                bool3=temp_graph.color_argument2(vertex1.index, temp_done_list, current_distance + 1, max_distance, max_depth)
                                if bool3==False:
                                    list4.append(vertex1.index)
                
                #####If there are no ways to force the color, return True #####
                if (len(list0 + list1 + list2 + list3 + list4) ==0):
                    return True
                #####If there is only one way to force that color, append that structure #####
                elif (len(list0 + list1 + list2 + list3 + list4) ==1):
                    #####If a neighbor is already colored with temp_color #####
                    if len(list0) > 0:
                        for vertex1 in self.vertex_list:
                            if vertex1.index==list0[0]:
                                if self.color_argument2(vertex1.index, done_list + [start_vert], current_distance + 1, max_distance, max_depth):
                                    return True
                    #####Coloring a vertex #####
                    if len(list1) > 0:
                        for vertex1 in self.vertex_list:
                            if vertex1.index==list1[0]:
                                vertex1.color=temp_color
                                self.clean()
                                self.fill_in_colors()
                                self.check_for_triangles()
                                if self.color_argument2(vertex1.index, done_list + [start_vert], current_distance + 1, max_distance, max_depth):
                                    return True
                    #####Adding an edge and coloring a vertex #####
                    if len(list2) > 0:
                        for vertex1 in self.vertex_list:
                            if vertex1.index==list2[0]:
                                vertex1.color=temp_color
                                vertex1.edges.append(start_vert)
                                self.clean()
                                self.fill_in_colors()
                                self.check_for_triangles()
                                if self.color_argument2(vertex1.index, done_list + [start_vert], current_distance + 1, max_distance, max_depth):
                                    return True
                                
                    #####Appending a vertex to the graph, coloring it, and adding an edge #####
                    if len(list3) > 0:
                        temp_vert = vert([start_vert], [], max(self.index_list) + 1)
                        #print(temp_vert.index)
                        temp_vert.color=temp_color
                        self.vertex_list.append(temp_vert)
                        self.clean()
                        self.fill_in_colors()
                        self.check_for_triangles()
                        if self.color_argument2(temp_vert.index, done_list + [start_vert], current_distance + 1, max_distance, max_depth):
                            return True
                        
                    #####Adding an edge #####
                    if len(list4) > 0:
                        for vertex1 in self.vertex_list:
                            if vertex1.index==list4[0]:
                                vertex1.edges.append(start_vert)
                                self.clean()
                                self.fill_in_colors()
                                self.check_for_triangles()
                                if self.color_argument2(vertex1.index, done_list + [start_vert], current_distance + 1, max_distance, max_depth):
                                    return True
                    #####Uncommenting this line will show much more detailed output of the algorithm, although it is admittedly pretty difficult to read #####
                    print('vertex: ' + str(start_vert) + ', done_list: ' + str(done_list) + ', color: ' +  str(temp_color) + ', depth: ' +  str(self.depth) + ', ' + str([list0, list1, list2, list3, list4]))
        return False
    
    def color_argument2_iter(self, start_vert, max_distance):
        #####This method iteratively applies the prior method, checks for updates, as well as checking for forks between applying coloring arguments. #####
        #####Not constantly checking for forks in the coloring method improves the runtime substantially. #####
        #####This method hides a lot of the inputs for color_argument2 to make it more 'user friendly' #####
        keep_going=True
        k=0
        while keep_going:
            k=k+1
            print('Iteration: ' + str(k))
            print('There are ' + str(len(self.vertex_list)) + ' vertices in the graph.')
            old_graph = copy.deepcopy(self)
            #####With the changes to the code, there is not much meaningful difference between the distance and depth. I can probably remove the second variable in the next update but I wanted to get this update out sooner. #####
            if self.color_argument2(start_vert, [], 0, max_distance, max_distance):
                return True
            if self.check_for_forks_triangles_iter(1):
                return True
            #####Check for updates
            updated =False
            if len(self.vertex_list)!=len(old_graph.vertex_list):
                updated=True
            if updated==False:
                for vertex1 in self.vertex_list:
                    for vertex2 in old_graph.vertex_list:
                        if (vertex1.index==vertex2.index) and (vertex1.color !=vertex2.color):
                            updated=True
                        if (vertex1.index==vertex2.index) and (vertex1.edges !=vertex2.edges):
                            updated=True
                        if (vertex1.index==vertex2.index) and (vertex1.non_edges !=vertex2.non_edges):
                            updated=True
            if updated==False:
                keep_going=False
        return False
                

In [131]:
v_0 = vert([1, 4], [], 0)
v_1 = vert([2, 0], [], 1)
v_2 = vert([3, 1], [], 2)
v_3 = vert([4, 2], [], 3)
v_4 = vert([0, 3], [], 4)
C_5 = graph([v_0, v_1, v_2, v_3, v_4])
C_5.clean()
C_5.show()
print(C_5.contains_triangle())
print(C_5.contains_fork())

Vertex 0:
    Edges:     [1, 4]
    Non-edges: []
    Unknown:   [2, 3]
    Color:   None
Vertex 1:
    Edges:     [2, 0]
    Non-edges: []
    Unknown:   [3, 4]
    Color:   None
Vertex 2:
    Edges:     [3, 1]
    Non-edges: []
    Unknown:   [0, 4]
    Color:   None
Vertex 3:
    Edges:     [4, 2]
    Non-edges: []
    Unknown:   [0, 1]
    Color:   None
Vertex 4:
    Edges:     [0, 3]
    Non-edges: []
    Unknown:   [1, 2]
    Color:   None
False
False


In [132]:
v_0 = vert([1, 2, 3, 4], [5, 6], 0)
v_1 = vert([0, 5], [2, 3, 4, 6], 1)
v_2 = vert([0, 6], [1, 3, 4, 5], 2)
v_3 = vert([0], [1, 2, 4, 5, 6], 3)
v_4 = vert([0], [1, 2, 3, 5, 6], 4)
v_5 = vert([1], [0, 2, 3, 4, 6], 5)
v_6 = vert([2], [0, 1, 3, 4, 5], 6)
fork = graph([v_0, v_1, v_2, v_3, v_4, v_5, v_6])
fork.clean()

v_0 = vert([1, 2, 3, 4], [5, 6], 0)
v_1 = vert([0, 5], [2, 3, 4, 6], 1)
v_2 = vert([0, 6], [1, 3, 4, 5], 2)
v_3 = vert([0], [1, 2, 4, 5], 3)
v_4 = vert([0], [1, 2, 3, 5, 6], 4)
v_5 = vert([1], [0, 2, 3, 4, 6], 5)
v_6 = vert([2], [0, 1, 4, 5], 6)
almost_fork = graph([v_0, v_1, v_2, v_3, v_4, v_5, v_6])
almost_fork.clean()
almost_fork.show()

Vertex 0:
    Edges:     [1, 2, 3, 4]
    Non-edges: [5, 6]
    Unknown:   []
    Color:   None
Vertex 1:
    Edges:     [0, 5]
    Non-edges: [2, 3, 4, 6]
    Unknown:   []
    Color:   None
Vertex 2:
    Edges:     [0, 6]
    Non-edges: [1, 3, 4, 5]
    Unknown:   []
    Color:   None
Vertex 3:
    Edges:     [0]
    Non-edges: [1, 2, 4, 5]
    Unknown:   [6]
    Color:   None
Vertex 4:
    Edges:     [0]
    Non-edges: [1, 2, 3, 5, 6]
    Unknown:   []
    Color:   None
Vertex 5:
    Edges:     [1]
    Non-edges: [0, 2, 3, 4, 6]
    Unknown:   []
    Color:   None
Vertex 6:
    Edges:     [2]
    Non-edges: [0, 1, 4, 5]
    Unknown:   [3]
    Color:   None


In [133]:
print(almost_fork.contains_triangle())
print(almost_fork.contains_fork())
print(fork.contains_triangle())
print(fork.contains_fork())

False
False
False
True


In [134]:
almost_fork.depth=0
if almost_fork.check_for_forks_triangles(1):
    print('Contradiction!')
almost_fork.show()

Vertex 0:
    Edges:     [1, 2, 3, 4]
    Non-edges: [5, 6]
    Unknown:   []
    Color:   None
Vertex 1:
    Edges:     [0, 5]
    Non-edges: [2, 3, 4, 6]
    Unknown:   []
    Color:   None
Vertex 2:
    Edges:     [0, 6]
    Non-edges: [1, 3, 4, 5]
    Unknown:   []
    Color:   None
Vertex 3:
    Edges:     [0, 6]
    Non-edges: [1, 2, 4, 5]
    Unknown:   []
    Color:   None
Vertex 4:
    Edges:     [0]
    Non-edges: [1, 2, 3, 5, 6]
    Unknown:   []
    Color:   None
Vertex 5:
    Edges:     [1]
    Non-edges: [0, 2, 3, 4, 6]
    Unknown:   []
    Color:   None
Vertex 6:
    Edges:     [2, 3]
    Non-edges: [0, 1, 4, 5]
    Unknown:   []
    Color:   None


In [135]:
v_0 = vert([1, 4,  5, 6], [2, 3], 0)
v_1 = vert([2, 0], [3, 4], 1)
v_2 = vert([3, 1, 6], [4, 0], 2)
v_3 = vert([4, 2], [0, 1], 3)
v_4 = vert([0, 3], [1, 2], 4)
v_5 = vert([0, 7], [8], 5)
v_6 = vert([0, 8], [], 6)
v_7 = vert([5], [0, 1, 2, 3, 4], 7)
v_8 = vert([6], [0, 1, 2, 3, 4], 8)
test_graph = graph([v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8])
test_graph.clean()
test_graph.depth=0
test_graph.show()
test_graph.check_for_forks_triangles_iter(2)
test_graph.show()

Vertex 0:
    Edges:     [1, 4, 5, 6]
    Non-edges: [2, 3, 7, 8]
    Unknown:   []
    Color:   None
Vertex 1:
    Edges:     [2, 0]
    Non-edges: [3, 4, 7, 8]
    Unknown:   [5, 6]
    Color:   None
Vertex 2:
    Edges:     [3, 1, 6]
    Non-edges: [4, 0, 7, 8]
    Unknown:   [5]
    Color:   None
Vertex 3:
    Edges:     [4, 2]
    Non-edges: [0, 1, 7, 8]
    Unknown:   [5, 6]
    Color:   None
Vertex 4:
    Edges:     [0, 3]
    Non-edges: [1, 2, 7, 8]
    Unknown:   [5, 6]
    Color:   None
Vertex 5:
    Edges:     [0, 7]
    Non-edges: [8]
    Unknown:   [1, 2, 3, 4, 6]
    Color:   None
Vertex 6:
    Edges:     [0, 8, 2]
    Non-edges: []
    Unknown:   [1, 3, 4, 5, 7]
    Color:   None
Vertex 7:
    Edges:     [5]
    Non-edges: [0, 1, 2, 3, 4]
    Unknown:   [6, 8]
    Color:   None
Vertex 8:
    Edges:     [6]
    Non-edges: [0, 1, 2, 3, 4, 5]
    Unknown:   [7]
    Color:   None
Vertex 0:
    Edges:     [1, 4, 5, 6]
    Non-edges: [2, 3, 7, 8]
    Unknown:   []
    Color:  

In [136]:
v_0 = vert([1, 4, 5, 6], [], 0)
v_1 = vert([2, 0], [], 1)
v_2 = vert([3, 1, 6], [], 2)
v_3 = vert([4, 2], [], 3)
v_4 = vert([0, 3], [], 4)
v_5 = vert([0, 7], [8], 5)
v_6 = vert([0, 8], [], 6)
v_7 = vert([5], [0, 1, 2, 3, 4], 7)
v_8 = vert([6], [0, 1, 2, 3, 4], 8)
test_graph = graph([v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8])
test_graph.clean()
test_graph.depth=0
test_graph.show()
test_graph.fill_in_graph(2)
test_graph.show()

Vertex 0:
    Edges:     [1, 4, 5, 6]
    Non-edges: [7, 8]
    Unknown:   [2, 3]
    Color:   None
Vertex 1:
    Edges:     [2, 0]
    Non-edges: [7, 8]
    Unknown:   [3, 4, 5, 6]
    Color:   None
Vertex 2:
    Edges:     [3, 1, 6]
    Non-edges: [7, 8]
    Unknown:   [0, 4, 5]
    Color:   None
Vertex 3:
    Edges:     [4, 2]
    Non-edges: [7, 8]
    Unknown:   [0, 1, 5, 6]
    Color:   None
Vertex 4:
    Edges:     [0, 3]
    Non-edges: [7, 8]
    Unknown:   [1, 2, 5, 6]
    Color:   None
Vertex 5:
    Edges:     [0, 7]
    Non-edges: [8]
    Unknown:   [1, 2, 3, 4, 6]
    Color:   None
Vertex 6:
    Edges:     [0, 8, 2]
    Non-edges: []
    Unknown:   [1, 3, 4, 5, 7]
    Color:   None
Vertex 7:
    Edges:     [5]
    Non-edges: [0, 1, 2, 3, 4]
    Unknown:   [6, 8]
    Color:   None
Vertex 8:
    Edges:     [6]
    Non-edges: [0, 1, 2, 3, 4, 5]
    Unknown:   [7]
    Color:   None
Vertex 0:
    Edges:     [1, 4, 5, 6]
    Non-edges: [7, 8, 2, 3, 9]
    Unknown:   []
    Color: 

In [137]:
v_9 = vert([3], [], 10)
test_graph.vertex_list.append(v_9)
test_graph.clean()
test_graph.fill_in_graph(1)
test_graph.show()

Vertex 0:
    Edges:     [1, 4, 5, 6]
    Non-edges: [7, 8, 2, 3, 9]
    Unknown:   [10]
    Color:   None
Vertex 1:
    Edges:     [2, 0, 9]
    Non-edges: [7, 8, 3, 4, 5, 6]
    Unknown:   [10]
    Color:   None
Vertex 2:
    Edges:     [3, 1, 6]
    Non-edges: [7, 8, 0, 4, 5, 9, 10]
    Unknown:   []
    Color:   None
Vertex 3:
    Edges:     [4, 2, 5, 10]
    Non-edges: [7, 8, 0, 1, 6]
    Unknown:   [9]
    Color:   None
Vertex 4:
    Edges:     [0, 3]
    Non-edges: [7, 8, 1, 2, 5, 6, 10]
    Unknown:   [9]
    Color:   None
Vertex 5:
    Edges:     [0, 7, 3]
    Non-edges: [8, 1, 4, 6, 2, 10]
    Unknown:   [9]
    Color:   None
Vertex 6:
    Edges:     [0, 8, 2]
    Non-edges: [1, 3, 4, 5, 9]
    Unknown:   [7, 10]
    Color:   None
Vertex 7:
    Edges:     [5]
    Non-edges: [0, 1, 2, 3, 4]
    Unknown:   [6, 8, 9, 10]
    Color:   None
Vertex 8:
    Edges:     [6]
    Non-edges: [0, 1, 2, 3, 4, 5]
    Unknown:   [7, 9, 10]
    Color:   None
Vertex 9:
    Edges:     [1]
    No

In [138]:
for vertex in C_5.vertex_list:
    vertex.max_degree = 3
v_5 = vert([0, 2], [], 5)
C_5.vertex_list.append(v_5)
C_5.clean()
C_5.show()

Vertex 0:
    Edges:     [1, 4, 5]
    Non-edges: [0, 2, 3]
    Unknown:   []
    Color:   None
Vertex 1:
    Edges:     [2, 0]
    Non-edges: []
    Unknown:   [3, 4, 5]
    Color:   None
Vertex 2:
    Edges:     [3, 1, 5]
    Non-edges: [0, 2, 4]
    Unknown:   []
    Color:   None
Vertex 3:
    Edges:     [4, 2]
    Non-edges: [0]
    Unknown:   [1, 5]
    Color:   None
Vertex 4:
    Edges:     [0, 3]
    Non-edges: [2]
    Unknown:   [1, 5]
    Color:   None
Vertex 5:
    Edges:     [0, 2]
    Non-edges: []
    Unknown:   [1, 3, 4]
    Color:   None


In [139]:
C_5.fill_in_colors(0)
C_5.show()

Vertex 0:
    Edges:     [1, 4, 5]
    Non-edges: [0, 2, 3]
    Unknown:   []
    Color:   None
Vertex 1:
    Edges:     [2, 0]
    Non-edges: []
    Unknown:   [3, 4, 5]
    Color:   0
Vertex 2:
    Edges:     [3, 1, 5]
    Non-edges: [0, 2, 4]
    Unknown:   []
    Color:   1
Vertex 3:
    Edges:     [4, 2]
    Non-edges: [0]
    Unknown:   [1, 5]
    Color:   None
Vertex 4:
    Edges:     [0, 3]
    Non-edges: [2]
    Unknown:   [1, 5]
    Color:   1
Vertex 5:
    Edges:     [0, 2]
    Non-edges: []
    Unknown:   [1, 3, 4]
    Color:   2


In [140]:
print(C_5.color_argument2_iter(0, 3))

Iteration: 1
There are 6 vertices in the graph.
vertex: 2, done_list: [0, 1], color: 0, depth: 2, [[], [3], [], [], []]
vertex: 2, done_list: [0, 1], color: 2, depth: 2, [[5], [], [], [], []]
vertex: 2, done_list: [0, 1], color: 0, depth: 1, [[], [3], [], [], []]
vertex: 2, done_list: [0, 1], color: 2, depth: 1, [[5], [], [], [], []]
vertex: 1, done_list: [0], color: 1, depth: 1, [[2], [], [], [], []]
vertex: 1, done_list: [0], color: 2, depth: 1, [[], [], [], [6], []]
vertex: 2, done_list: [0, 1], color: 0, depth: 1, [[], [3], [], [], []]
vertex: 2, done_list: [0, 1], color: 2, depth: 1, [[5], [], [], [], []]
vertex: 2, done_list: [0, 1], color: 0, depth: 0, [[], [3], [], [], []]
vertex: 2, done_list: [0, 1], color: 2, depth: 0, [[5], [], [], [], []]
vertex: 1, done_list: [0], color: 1, depth: 0, [[2], [], [], [], []]
vertex: 1, done_list: [0], color: 2, depth: 0, [[], [], [], [6], []]
vertex: 0, done_list: [], color: 0, depth: 0, [[1], [], [], [], []]
vertex: 4, done_list: [0], color

In [141]:
C_5.show()

Vertex 0:
    Edges:     [1, 4, 5]
    Non-edges: [0, 2, 3, 6, 7, 8, 9]
    Unknown:   []
    Color:   None
Vertex 1:
    Edges:     [2, 0, 6]
    Non-edges: [1, 3, 4, 5, 7, 8, 9]
    Unknown:   []
    Color:   0
Vertex 2:
    Edges:     [3, 1, 5]
    Non-edges: [0, 2, 4, 6, 7, 8, 9]
    Unknown:   []
    Color:   1
Vertex 3:
    Edges:     [4, 2]
    Non-edges: [0, 1, 5, 7, 6]
    Unknown:   [8, 9]
    Color:   0
Vertex 4:
    Edges:     [0, 3, 7]
    Non-edges: [1, 2, 4, 5, 6, 8, 9]
    Unknown:   []
    Color:   1
Vertex 5:
    Edges:     [0, 2, 7, 9]
    Non-edges: [1, 3, 4, 6, 8]
    Unknown:   []
    Color:   2
Vertex 6:
    Edges:     [1]
    Non-edges: [0, 2, 5, 3, 4]
    Unknown:   [7, 8, 9]
    Color:   2
Vertex 7:
    Edges:     [5, 8, 4]
    Non-edges: [0, 1, 2, 3, 9]
    Unknown:   [6]
    Color:   0
Vertex 8:
    Edges:     [7]
    Non-edges: [0, 1, 2, 4, 5, 9]
    Unknown:   [3, 6]
    Color:   1
Vertex 9:
    Edges:     [5]
    Non-edges: [0, 1, 2, 4, 8, 7]
    Unknown:

In [144]:
v_0 = vert([1, 4, 5, 6], [], 0)
v_1 = vert([2, 0], [], 1)
v_2 = vert([3, 1], [], 2)
v_3 = vert([4, 2], [], 3)
v_4 = vert([0, 3], [], 4)
v_5 = vert([0, 7], [], 5)
v_6 = vert([0, 8], [], 6)
v_7 = vert([5, 8], [0, 1, 2, 3, 4], 7)
v_8 = vert([6, 7], [0, 1, 2, 3, 4], 8)
test_graph = graph([v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8])
test_graph.clean()
test_graph.check_for_forks_triangles_iter(1)
test_graph.show()

Vertex 0:
    Edges:     [1, 4, 5, 6]
    Non-edges: [7, 8, 2, 3]
    Unknown:   []
    Color:   None
Vertex 1:
    Edges:     [2, 0]
    Non-edges: [7, 8, 3, 4, 5, 6]
    Unknown:   []
    Color:   None
Vertex 2:
    Edges:     [3, 1]
    Non-edges: [7, 8, 0, 4]
    Unknown:   [5, 6]
    Color:   None
Vertex 3:
    Edges:     [4, 2]
    Non-edges: [7, 8, 0, 1]
    Unknown:   [5, 6]
    Color:   None
Vertex 4:
    Edges:     [0, 3]
    Non-edges: [7, 8, 1, 2, 5, 6]
    Unknown:   []
    Color:   None
Vertex 5:
    Edges:     [0, 7]
    Non-edges: [1, 4, 6, 8]
    Unknown:   [2, 3]
    Color:   None
Vertex 6:
    Edges:     [0, 8]
    Non-edges: [1, 4, 5, 7]
    Unknown:   [2, 3]
    Color:   None
Vertex 7:
    Edges:     [5, 8]
    Non-edges: [0, 1, 2, 3, 4, 6]
    Unknown:   []
    Color:   None
Vertex 8:
    Edges:     [6, 7]
    Non-edges: [0, 1, 2, 3, 4, 5]
    Unknown:   []
    Color:   None


In [145]:
print(test_graph.get_fork_edge_lists())

[[[5, 2], [6, 2]], [[6, 2], [5, 2]], [[5, 3], [6, 3]], [[6, 3], [5, 3]]]
